In [ ]:
  %%html
  <script>
      // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
      require(
          ['base/js/namespace', 'jquery'], 
          function(jupyter, $) {
              $(jupyter.events).on("kernel_ready.Kernel", function () {
                  console.log("Auto-running all cells-below...");
                  jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                  jupyter.actions.call('jupyter-notebook:save-notebook');
              });
          }
      );
  </script>


In [ ]:
  %%html
  <script>
      code_show=true; 
      function code_toggle() {
          if (code_show){
              $('div.input').hide();
          } else {
              $('div.input').show();
          }
          code_show = !code_show
      } 
      $( document ).ready(code_toggle);
  </script>
  The raw code for this IPython notebook is by default hidden for easier reading.
  To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


In [ ]:
  %%html
  <script>
      $([IPython.events]).on("kernel_ready.Kernel", function () {
          $('div#header-container').hide();
          $('div#maintoolbar').hide();
      });
  </script>


In [ ]:
# first cell used to Show the code (useful for Debug)
from IPython.display import HTML
HTML('''<script>
 $('div.input').show();
</script>
''')

In [ ]:
# Import the needed librairies 
from IPython.display import display

# Do not show pip install info
!python3 -m pip install qarnot > /dev/null
import qarnot 
import ipywidgets as widgets
from tkinter import Tk, filedialog
from IPython.display import clear_output, display

In [ ]:
# Hide the code 
from IPython.display import HTML

HTML('''<script>
$('div.input').hide();
</script>''')

In [ ]:
from ipywidgets import Layout, Button, Box, Label, BoundedIntText, IntSlider, Dropdown, SelectMultiple

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)
percent50layout = Layout(width='70%', height='50px')

# Simswap Demo

## Enter your information

### Enter your Qarnot token

In [ ]:
prepare_box = [
    ("pwd", "Qarnot Token", widgets.Password(placeholder='Enter your token', disabled=False, layout=percent50layout)),
    ("name", "Task name", widgets.Text(placeholder='Type something', disabled=False, value="simswap", layout=percent50layout)),
    ("result", "Bucket result name", widgets.Text(placeholder='Type something', disabled=False, value="simswap-result", layout=percent50layout)),
    ("ressource", "Bucket resources name", widgets.Text(placeholder='Type something', disabled=False, value="simswap-ressources", layout=percent50layout)),
]

qarnot_items = {}
for key, label, widget in prepare_box:
    qarnot_items[key] = Box([Label(value=label),widget], layout=form_item_layout) 
    display(qarnot_items[key]) 

### Enter the picture and video to be swaped
If no picture or no video is set, the default simswap demo picture and video will be used.  

* For a remote binder usage upload your files,  
* For a local usage give you path files names.  
  
#### Upload files, using binder and the remote files :

In [ ]:
resources_items = {}
prepare_resource_box = [
    ("upload_picture", "Face picture image", widgets.FileUpload(accept='.jpg',multiple=False, layout=percent50layout)),
    ("upload_video", "Video to be trasformed", widgets.FileUpload(accept='.mp4',multiple=False, layout=percent50layout)),
]


for key, label, widget in prepare_resource_box:
    resources_items[key] = Box([Label(value=label),widget], layout=form_item_layout) 
    display(resources_items[key]) 

#### Get local files, running your ipynb locally :

In [ ]:
prepare_resource_box = [
    ("picture", "Face picture image", widgets.Text(placeholder='Type your path', disabled=False, layout=percent50layout)),
    ("video", "Video to be trasformed", widgets.Text(placeholder='Type your path', disabled=False, layout=percent50layout)),
]

for key, label, widget in prepare_resource_box:
    resources_items[key] = Box([Label(value=label),widget], layout=form_item_layout) 
    display(resources_items[key]) 

## Run the program

The result video is called "simswap_result_multi.mp4"   

You will find it in the jupyter root folder or in the binder files.   


In [ ]:
bash_script = """
(
    I=0; 
    while true ; 
    do 
        echo $I; 
         sleep 120; 
         nvidia-smi>>nvidia.log; 
    done;
)& 
echo $IMAGE_PATH_MAIN
echo $VIDEO_PATH

cat /app/demo-test.sh
bash /app/demo-test.sh
python test_video_swapmulti.py --isTrain false --use_mask --name people --Arc_path arcface_model/arcface_checkpoint.tar --pic_a_path ${IMAGE_PATH_MAIN} --video_path ${VIDEO_PATH} --output_path ${DOCKER_WORKDIR}/simswap_result_multi.mp4 --temp_path ./temp_results  > stdout.log 2> stderr.log

echo end

jobs -p | xargs kill
exit 0 ;
"""

In [ ]:
import os
import io
button_layout =  Layout(
    margin="auto",
    width='70%',
    height='50px'
)
check_values_layout = {'border': '1px solid black', "margin":"auto","padding":"15px", "text-size":"20px", "font-weight":"20px"}
check_values_layout_box = Layout(width="100%")


button = widgets.Button(description="Verify the computation launch!",
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check', 
    layout=button_layout)
output = widgets.Output()

def get_launch_button():
    launch_button = widgets.Button(description="Send the task!",
        disabled=False,
        button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Click me',
        icon='check',
        layout=button_layout)
    launch_script = widgets.Output()
    box_second_button = Box([launch_button], layout=form_item_layout)
    return launch_button, launch_script, box_second_button
launch_button, launch_script, box_second_button = get_launch_button()

box_button = Box([button], layout=form_item_layout) 
display(box_button, output, layout=button_layout)

def get_upload_file(file_name, file_content):
    with open(file_name, 'wb') as output_file: 
        for uploaded_filename in file_content:
            content = file_content[uploaded_filename]['content']   
            output_file.write(content) 

values = {}
def launch(b):
    with launch_script:
        # Prepare the data
        image_name = None
        video_name = None
        if values["upload_picture"]:
            get_upload_file("picture.jpg", values["upload_picture"])
            values["picture"] = "picture.jpg"
            image_name = "picture.jpg"
        elif values["picture"]:
            image_name = os.path.basename(values["picture"])
        if values["upload_video"]:
            get_upload_file("video.mp4", values["upload_video"])
            values["video"] = "video.mp4"
            video_name = "video.mp4"
        elif values["video"]:
            video_name = os.path.basename(values["video"])
        
        pwd_value = values["pwd"]
        name_value = values["name"]
        
        cluster_url = "https://api.qarnot.com"
        instances = 1
        profile = "docker-nvidia-bi-a6000-batch"
        
        docker_repo = 'guillaumenebieqarnot/simswap'
        docker_tag = 'latest'
        docker_cmd = "bash run.sh"
        global bash_script 
        
        launch_button.on_click(launch)
        # prepare the bucket resources

        # Run the data
        conn = qarnot.connection.Connection(client_token=pwd_value, cluster_url=cluster_url)
        bucket_resources = conn.retrieve_or_create_bucket(values["ressource"])
        if image_name: 
            bash_script = "export IMAGE_PATH_MAIN=/job/" + image_name + ";" + bash_script
            bucket_resources.add_file(values["picture"], image_name)
        if video_name:  
            bucket_resources.add_file(values["video"], video_name)
            bash_script = "export VIDEO_PATH=/job/" + video_name + ";" + bash_script
        bucket_resources.add_file(io.BytesIO(bytes(bash_script, 'utf-8')), "run.sh")
                                  
        resources_value = [ bucket_resources ]
        bucket_result = conn.retrieve_or_create_bucket(values["result"])
        result_value = bucket_result

        task = conn.create_task(name_value, profile, instances)
        task.constants['DOCKER_REPO'] = docker_repo
        task.constants['DOCKER_TAG'] = docker_tag
        task.constants['DOCKER_CMD'] = docker_cmd
        task.snapshot(180)
        task.results = result_value
        task.resources = resources_value
        print("*** task launched ***")
        task.run()
        print("Task result " + task.state)
        bucket_result.get_all_files(".")
        print("result downloaded" + str([object_.key for object_ in bucket_result.list_files()]) )
        display(FileLink("simswap_result_multi.mp4"))
        
        # retrieve the bucket result
out_values = widgets.Output(layout=check_values_layout)
out_box = Box([out_values], layout=check_values_layout_box)
def on_button_clicked(b):
    with output:
        # Now the key: the container is displayed (while empty) in the main thread
        display(out_box)
        box_second_button = Box([launch_button], layout=form_item_layout) 
        
        for key in qarnot_items:
            values[key] = qarnot_items[key].children[1].value
        for key in resources_items:
            values[key] = resources_items[key].children[1].value
        
        with out_values:
            if not values["pwd"]:
                print("No token found ! the task can't run.")
                return
            print("Launch values: ")
            print("Name: " + values["name"])
            if values["upload_picture"]:
                print("Remote File: " + str(list(values["upload_picture"].keys())))
            else:
                print("File path: " + values["picture"])
            if values["upload_video"]:
                print("Remote Video: " + str(list(values["upload_video"].keys())))
            else:
                print("Video path: " + values["video"])
        display(box_second_button, launch_script)
        launch_button.on_click(launch)

button.on_click(on_button_clicked)